In [ ]:
import numpy as np

from Related_Codes.DCNAutoencoder import DCNAutoencoder, DCNCDAutoencoder
from Related_Codes.DECAutoencoder import DECAutoencoder, DECCDAutoencoder
from Related_Codes.IDECAutoencoder import IDECAutoencoder, IDECCDAutoencoder
from General_Functions import General_Functions

from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.manifold import LocallyLinearEmbedding, SpectralEmbedding, Isomap, TSNE

import torch

import os
import sys

from Datasets.Datasets_Functions import *
from Visualization import Visualization

In [ ]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

In [ ]:
hashmap_path = path_to_module + 'Datasets/'
hashmap = get_hashmap(hashmap_path)
print(hashmap)
dataset_name = "emnist_balanced_digits"
dataset_properties = hashmap[dataset_name]
batch_size = dataset_properties['batch_size'] = 256
n_clusters = dataset_properties['n_clusters'] = 10
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, dataset_properties)
print(input_dim)

In [ ]:
visualization = Visualization()

### Dcn

In [ ]:

"""
parser.add_argument('--n-classes', type=int, default=10,
                    help='output dimension')
"""
n_clusters = dataset_properties['n_clusters'] = 10

# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size'] = 256

# Pre-Training Epochs and Learning Rate
"""
parser.add_argument('--pre-epoch', type=int, default=50, 
                    help='number of pre-train epochs')
"""
n_pret_epochs = 100
pret_lr = 1e-3

# Lamdas, Training Epochs and Learning Rate
"""
parser.add_argument('--epoch', type=int, default=100,
                    help='number of epochs to train')
"""
n_epochs = 100
"""
parser.add_argument('--lr', type=float, default=1e-4,
                    help='learning rate (default: 1e-4)')
"""
lr = 1e-4
"""
parser.add_argument('--lamda', type=float, default=1,
                    help='coefficient of the reconstruction loss')
"""
lamda = 1
"""
parser.add_argument('--beta', type=float, default=1,
                    help=('coefficient of the regularization term on '
                          'clustering'))
"""
beta = 1e-5
"""
parser.add_argument('--latent_dim', type=int, default=10,
                    help='latent space dimension')
"""
latent_dim = 10
n_channels = 1
is_MLP_AE = False

if(is_MLP_AE):
    input_dim = 784
    dcn_autoencoder = DCNAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim)
else:  
    input_dim = 1
    dcn_autoencoder = DCNCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, n_channels=n_channels) 
dcn_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
dcn_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
dcn_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, lamda=lamda, beta=beta )
dcn_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
dcn_autoencoder.set_path()
dcn_autoencoder = dcn_autoencoder.to(device)
dcn_autoencoder
pretrain = True
if(pretrain):
    dcn_autoencoder.pretrain_autoencoder()
    dcn_autoencoder.save_pretrained_weights()
else:
    model_save_path = dcn_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    dcn_autoencoder.load_state_dict(torch.load(model_save_path))

In [ ]:
dcn_autoencoder.train_autoencoder()
cluster_centers = dcn_autoencoder.get_cluster_centers()
_, dcn_autoencoder_reduced_data, labels = dcn_autoencoder.get_latent_data()
General_Functions().save_excel(dcn_autoencoder.data_dir_path, dcn_autoencoder.df_eval)
visualization.plot_tsne(dcn_autoencoder_reduced_data, labels, labels, cluster_centers, dcn_autoencoder.data_dir_path)